In [2]:
#importing the kaggle api library
!pip install kaggle
import kaggle
!kaggle datasets download -d ankitbansal06/retail-orders

  Using cached kaggle-1.7.4.2-py3-none-any.whl.metadata (16 kB)
Using cached kaggle-1.7.4.2-py3-none-any.whl (173 kB)
Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0
retail-orders.zip: Skipping, found more recently modified local copy (use --force to force download)


In [51]:
#extracting all things from zip archive
import zipfile
zip_ref = zipfile.ZipFile('retail-orders.zip')
zip_ref.extractall()
zip_ref.close()

In [3]:
#importing a dataframe, at the same time correcting the values in ship mode column
import pandas as pd
df = pd.read_csv("orders.csv", na_values = ['Not Available','unknown'])
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [4]:
#making a better formatting for the header
df.columns=df.columns.str.lower()
df.columns=df.columns.str.replace(' ','_')

In [5]:
#calculating the discount, sale price and marginality
df['discount']= df['list_price']*df['discount_percent']*.01
df['final_price']=df['list_price']-df['discount']
df['profit']=df['final_price']-df['cost_price']

In [6]:
#converting order date to datetime format
df['order_date']= pd.to_datetime(df['order_date'],format='%Y-%m-%d')

In [7]:
df.drop(columns=['cost_price','list_price','discount_percent'],inplace = True)

In [8]:
df.head(20)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,quantity,discount,final_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,2,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,5,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,2,1.0,19.0,-1.0
5,6,2022-03-13,NaN,Consumer,United States,Los Angeles,California,90032,West,Furniture,Furnishings,FUR-FU-10001487,7,1.5,48.5,-1.5
6,7,2022-12-28,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Art,OFF-AR-10002833,4,0.3,9.7,-0.3
7,8,2022-01-25,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Technology,Phones,TEC-PH-10002275,6,45.5,864.5,4.5
8,9,2023-03-23,NaN,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Binders,OFF-BI-10003910,3,0.4,19.6,-0.4
9,10,2023-05-16,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Appliances,OFF-AP-10002892,5,3.3,106.7,16.7


In [11]:
import sqlalchemy as sal
engine = sal.create_engine("mssql+pyodbc://Fike-PC/Project_DB?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes", echo=True)
conn = engine.connect()

2025-03-29 20:53:14,184 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2025-03-29 20:53:14,196 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-29 20:53:14,198 INFO sqlalchemy.engine.Engine SELECT schema_name()
2025-03-29 20:53:14,199 INFO sqlalchemy.engine.Engine [generated in 0.00108s] ()
2025-03-29 20:53:14,205 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2025-03-29 20:53:14,206 INFO sqlalchemy.engine.Engine [generated in 0.00071s] ()
2025-03-29 20:53:14,207 INFO sqlalchemy.engine.Engine SELECT 1 FROM fn_listextendedproperty(default, default, default, default, default, default, default)
2025-03-29 20:53:14,208 INFO sqlalchemy.engine.Engine [generated in 0.00056s] ()


In [15]:
df.to_sql('df_orders',con = conn, index = False, if_exists = 'append')

2025-03-29 21:14:28,272 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-29 21:14:28,276 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE ([INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) OR [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max))) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2025-03-29 21:14:28,277 INFO sqlalchemy.engine.Engine [cached since 1113s ago] ('BASE TABLE', 'VIEW', 'df_orders', 'dbo')
2025-03-29 21:14:28,369 INFO sqlalchemy.engine.Engine INSERT INTO df_orders (order_id, order_date, ship_mode, segment, country, city, state, postal_code, region, category, sub_category, product_id, quantity, discount, final_price, profit) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ? ... 6390 characters truncated ... , (?, ?, ?, ?, ?, ?, ?, ?,

38

In [17]:
df.head(20)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,quantity,discount,final_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,2,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,5,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,2,1.0,19.0,-1.0
5,6,2022-03-13,NaN,Consumer,United States,Los Angeles,California,90032,West,Furniture,Furnishings,FUR-FU-10001487,7,1.5,48.5,-1.5
6,7,2022-12-28,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Art,OFF-AR-10002833,4,0.3,9.7,-0.3
7,8,2022-01-25,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Technology,Phones,TEC-PH-10002275,6,45.5,864.5,4.5
8,9,2023-03-23,NaN,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Binders,OFF-BI-10003910,3,0.4,19.6,-0.4
9,10,2023-05-16,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Appliances,OFF-AP-10002892,5,3.3,106.7,16.7
